In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import mlflow
from sklearn.metrics import root_mean_squared_error
import os
import requests

Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

    load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
    fit a DictVectorizer on the training set (January 2023 data),
    save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

In [2]:
urls = ["https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"]
folder = "taxi_dataset"

os.makedirs(folder, exist_ok=True)

for url in urls:
    filename = url.split("/")[-1]
    output_path = os.path.join(folder, filename)

    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"File saved")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

File saved
File saved
File saved


In [3]:
!python preprocess_data.py --raw_data_path ./taxi_dataset --dest_path ./output

Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

    load the datasets produced by the previous step,
    train the model on the training set,
    calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

In [4]:
!python train.py

2025/07/30 11:47:22 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-rf' does not exist. Creating a new experiment.
2025/07/30 11:47:22 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

    launch the tracking server on your local machine,
    select a SQLite db for the backend store and a folder called artifacts for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In [5]:
mlflow.set_tracking_uri("sqlite::///mlflow.db")

Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

    the list of hyperparameters that are passed to the objective function during the optimization,
    the RMSE obtained on the validation set (February 2023 data).

In [6]:
!python hpo.py

2025/07/30 11:59:35 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run caring-shrike-323 at: http://127.0.0.1:5000/#/experiments/1/runs/19c6c7725d714fbd92b7c248126bdb8a

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run sneaky-ape-475 at: http://127.0.0.1:5000/#/experiments/1/runs/383fbf98a3d141daa2a792a239dea033

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run orderly-colt-261 at: http://127.0.0.1:5000/#/experiments/1/runs/5c1b866cbcfe445e99108fa1e1e88f40

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run peaceful-mink-316 at: http://127.0.0.1:5000/#/experiments/1/runs/6c41478d705c44da94a500e23e1d48e4

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run monumental-conch-669 at: http://127.0.0.1:5000/#/experiments/1/runs/d6df971c9c2d4de9b7d8596

Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

In [10]:
!python register_model.py

2025/07/30 13:05:42 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
🏃 View run judicious-hawk-889 at: http://127.0.0.1:5000/#/experiments/2/runs/4760df97bbc64e5499168695ce7c99f3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run resilient-snake-933 at: http://127.0.0.1:5000/#/experiments/2/runs/3e8ea02cec29428689742eecf55aa8b0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run righteous-ape-311 at: http://127.0.0.1:5000/#/experiments/2/runs/1369e3fe43ed4a609c4296cf2d117a1c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run resilient-duck-858 at: http://127.0.0.1:5000/#/experiments/2/runs/2c70c6d7e6334af7aa77ea3309a5219c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2